In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
plt.rc("font", family = 'Malgun Gothic')
plt.rc("axes", unicode_minus=False)

## 크롤링한 파일 로드

In [4]:
df = pd.read_csv("inflearn-event.csv")
df.shape

(100, 1)

In [7]:
df.head()

,text
0,2020년 목표: 프알못에서 프로젝트 마스터 되기 \n관심 있는 강의 : 스프링 ...
1,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
2,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
3,2020년 목표 : 리액트 공부하기
4,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급


In [8]:
df.tail()

,text
95,"사이버수사대(디지털포렌식)를 지원하고 싶은 현직 경찰관입니다. 현재는 아는 것도, ..."
96,"퇴사한지 2년 (헉 벌써…) 되가는 백수 개발자입니다. 현재 리액트, 노드, 파이어..."
97,"취업 성공~~!!!\n#관심강의: 스프링, 자바, Swift"
98,"올해는 파이썬, 데이터 분석, 머신러닝을 배워\n업무에 조금이라도 활용하고 싶습니다..."
99,오늘코드 사랑해요 ~ \n데이터 분석 기초를 둔둔하게~ \n#관심강의 : Oracl...


## 데이터 전처리 

### 중복 값 제거 

In [9]:
print(df.shape)
df = df.drop_duplicates(["text"], keep="last")
print(df.shape)

(100, 1)
(97, 1)


### 원본 따로 보존

In [11]:
df["original_text"] = df["text"]
df.head()

<ipython-input-11-a76c922448ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["original_text"] = df["text"]


,text,original_text
0,2020년 목표: 프알못에서 프로젝트 마스터 되기 \n관심 있는 강의 : 스프링 ...,2020년 목표: 프알못에서 프로젝트 마스터 되기 \n관심 있는 강의 : 스프링 ...
1,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
2,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는...","2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
4,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
5,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!


### 소문자 변환
파이썬은 대소문자를 구분하기 때문에 데이터 필터링을 위해 대문자를 모두 소문자로 변경

In [12]:
df["text"] = df["text"].str.lower()

<ipython-input-12-df0d3383638d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].str.lower()


In [13]:
df["text"] = df["text"].str.replace("python","파이썬").str.replace("pandas", "판다스").str.replace("javascript","자바스크립트").str.replace("java","자바").str.replace("react","리액트")

<ipython-input-13-d5041db38d16>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"] = df["text"].str.replace("python","파이썬").str.replace("pandas", "판다스").str.replace("javascript","자바스크립트").str.replace("java","자바").str.replace("react","리액트")


### 관심 강의 분리

In [15]:
df['course'] = df['text'].apply(lambda x: x.split("관심강의")[-1])
df['course'] = df['course'].apply(lambda x: x.split("관심 강의")[-1])
df['course'] = df['course'].apply(lambda x: x.split("관심 강좌")[-1])
df['course'] = df['course'].str.replace(":","")
df["course"].tail()

<ipython-input-15-945724d097e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['course'] = df['text'].apply(lambda x: x.split("관심강의")[-1])
<ipython-input-15-945724d097e5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['course'] = df['course'].apply(lambda x: x.split("관심 강의")[-1])
<ipython-input-15-945724d097e5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

95           디지털포렌식 자격증 2급 대비강좌
96        파이썬 데이터시각화 분석 실전 프로젝트
97               스프링, 자바, swift
98            파이썬, 데이터 분석, 머신러닝
99      oracle sql, 데이터 분석, 파이썬
Name: course, dtype: object

In [16]:
df[['text','course']].tail()

,text,course
95,"사이버수사대(디지털포렌식)를 지원하고 싶은 현직 경찰관입니다. 현재는 아는 것도, ...",디지털포렌식 자격증 2급 대비강좌
96,"퇴사한지 2년 (헉 벌써…) 되가는 백수 개발자입니다. 현재 리액트, 노드, 파이어...",파이썬 데이터시각화 분석 실전 프로젝트
97,"취업 성공~~!!!\n#관심강의: 스프링, 자바, swift","스프링, 자바, swift"
98,"올해는 파이썬, 데이터 분석, 머신러닝을 배워\n업무에 조금이라도 활용하고 싶습니다...","파이썬, 데이터 분석, 머신러닝"
99,오늘코드 사랑해요 ~ \n데이터 분석 기초를 둔둔하게~ \n#관심강의 : oracl...,"oracle sql, 데이터 분석, 파이썬"


In [ ]:
search_keyword = ['머신러닝', '딥러닝', '파이썬', '판다스', '공공데이터', 'django', '크롤링', '시각화', '데이터 분석', ]